   ![alt text](https://www.mbari.org/wp-content/uploads/2014/11/logo-mbari-3b.png "MBARI")

  <div align="left">Copyright (c) 2021, MBARI</div>
    
  * Distributed under the terms of the GPL License
  * Maintainer: dcline@mbari.org
  * Authors: Danelle Cline dcline@mbari.org, John Ryan ryjo@mbari.org

## Blue whale song
---
Baleen whales produce rhythmic repeated sequences of sound; they sing.  Analysis of song occurrence patterns can yield insights into the behavioral ecology of blue whales.  This tutorial describes use of the *Pacific Ocean Sound Recordings* archive to examine temporal patterns in the occurrence of blue whale song within foraging habitat of the central California Current System.  Signal processing methods in this tutorial focus on the blue whale B call.  A companion tutorial illustrates classification of blue whale A calls using supervised machine learning.


## Install dependencies
---
Install dependencies and include all packages used in this tutorial. This only needs to be done once for the duration of this notebook.

In [ ]:
%pip install -q boto3
%pip install -q soundfile
%pip install -q scipy
%pip install -q numpy
%pip install -q matplotlib
import boto3, botocore
from botocore import UNSIGNED
from botocore.client import Config
from six.moves.urllib.request import urlopen
import io
import scipy
from scipy import signal
import numpy as np
import soundfile as sf
import matplotlib.pyplot as plt
from scipy.stats import binned_statistic

## Data Access
---
This section covers file listing, metadata retrieval, and data loading.

### List files
Files are organized by year and month; list all of the files available for one month of one year. 

In [ ]:
s3 = boto3.client('s3',
    aws_access_key_id='',
    aws_secret_access_key='', 
    config=Config(signature_version=UNSIGNED))

year = "2017"
month = "11"
bucket = 'pacific-sound-2khz'

for obj in s3.list_objects_v2(Bucket=bucket, Prefix=f'{year}/{month}')['Contents']:
    print(obj['Key'])

### Load data
Read a single daily file.

In [ ]:
# read full-day of data
filename = 'MARS-20171101T000000Z-2kHz.wav'
bucket = 'pacific-sound-2khz'
key = f'{year}/{month}/{filename}'
url = f'https://{bucket}.s3.amazonaws.com/{key}'
print(f'Reading from {url}')
v, sample_rate = sf.read(io.BytesIO(urlopen(url).read()),dtype='float32') 
v = v*3   # convert scaled voltage to volts
nsec = (v.size)/sample_rate # number of seconds in vector
print(f'Read {nsec} seconds of data')

## A view of blue whale song
---
To understand the method of quantifying song occurrence using acoustic power analysis (following Širović et al. 2004; 2009), it is useful to first consider the attributes of blue whale song.  Songs of the northeast Pacific blue whale population include three types of calls: A, B, and C.  The B calls have the strongest intensity and are thus often used to characterize song occurrence.  

Analysis approaches include (1) detecting, classifying, and counting calls, and (2) quantifying the energy within the frequency band of the call, relative to that at background frequencies.  The first approach becomes difficult during periods when the whales chorus because the presence of overlapping calls thwarts distinction of individual calls.  The second approach can be applied consistently regardless of whether or not vocalizations overlap.  Application of this second approach to years of recordings, together with animal-borne metrics, revealed an [acoustic signature of blue whale migration](https://www.sciencedirect.com/science/article/pii/S0960982220313312).

Using the data just loaded, let's examine blue whale song structure.



In [ ]:
# Compute spectrogram 
w = scipy.signal.get_window('hann',sample_rate)
f, t, psd = scipy.signal.spectrogram(v, sample_rate,nperseg=sample_rate,noverlap=0,window=w,nfft=sample_rate)
# psd is power spectral density
sens = -177.9  # hydrophone sensitivity at 250 Hz (deployment 2)
psd = 10*np.log10(psd) - sens
print(f':: psd.shape           = {psd.shape}')
print(f':: f.size              = {f.size}')
print(f':: t.size              = {t.size}')

# Subset 30 minutes
starthour = 7
startsec = int((starthour)*3600 +1)
endsec = startsec+1800
psds = psd[:,startsec:endsec]
print(f':: subset of psd shape  = {psds.shape}')
# Plot
plt.figure(dpi=200, figsize = [9,3])
plt.imshow(psds,aspect='auto',origin='lower',vmin=45,vmax=105)
plt.plot([1, 1790],[39, 39],'w--')
plt.plot([1, 1790],[48, 48],'w--')
plt.colorbar(label='Spectrum level (dB re 1 $\mu$Pa$^2$/Hz)')
plt.ylim(8,1000)
plt.yscale('log')
plt.xlabel('Second of hour 07')
plt.ylabel('Frequency (Hz)')
plt.title('Baleen whale song, 01-Nov-2017')
plt.annotate("C",(1250,10),color='w')
plt.annotate("B",(1250,13.5),color='w')
plt.annotate("B$_2$",(1250,27),color='w')
plt.annotate("B$_3$",(1250,41),color='w')
plt.annotate("B$_4$",(1250,55),color='w')
plt.annotate("A",(1250,78),color='w')
plt.annotate("blue whale",(1250,110),color='w')
plt.annotate("fin whale",(400,20),color='w')
plt.annotate("humpback whale",(600,500),color='w')


In the spectrogram above, the labels align with frequency bands where the call energy occurs.  The third harmonic of the blue whale B call, within the frequency band defined by the dashed lines, carries the most energy in their songs and is the strongest signal for analysis.  Fin whale calls occurred throughout the time period shown, brief (< 1 second) pulses with energy between the fundamental and second harmonic of the blue whale B calls.  Humpback whale song occupies the largest frequency range.

## Call index
---
To consider the peak and background frequencies used in the calculation of call index, let's examine the average spectrum levels for period shown above.

In [ ]:
m = np.mean(psds,axis=1)
plt.figure(dpi=200, figsize = [7,3])
xl = [74,98]
plt.plot(m,f,'k-o')
plt.ylim(33,52)
plt.xlim(xl)
plt.plot(xl,[43, 43],'b--')
plt.plot(xl,[35, 35],'r--')
plt.plot(xl,[50, 50],'r--')
plt.xlabel('Mean spectrum level (dB re 1 $\mu$Pa$^2$/Hz)')
plt.ylabel('Frequency (Hz)')

Power within the peak caused by B calls (dashed blue line) is much higher than that of the background (dashed red lines).  The call index (CI) is simply the ratio of signal (spectrum level at the peak frequency) to noise (average spectrum level at the background frequencies).  To examine response of the index to variation in call activity, let's compute CI at 1-second resolution.

In [ ]:
print(f':: subset of psd shape  = {psds.shape}')
pk = np.squeeze(psds[f==43,:])
print(f':: pk shape  = {pk.shape}')
bg = np.mean(np.squeeze(np.array(psds[(f==35) | (f==50),:])),axis=0); 
print(f':: bg shape  = {bg.shape}')
# Call index (CI)
CI = pk/bg; 
# reduce noise with a median filter (calls are ~10 s or longer)
CIf = scipy.signal.medfilt(CI,3)
plt.figure(dpi=200, figsize = [9,3])
plt.plot(CIf)
plt.xlim(0,1000)
plt.xlabel('Seconds')
plt.ylabel('CI')

##Time-series analysis
---
Now that we have methods to compute the call index, we can execute batch processing to extract time-series statistics.  A consideration here is that blue whale B calls have a relatively long duration, thus computing power spectral density over periods longer than 1 second (above) can be useful for data reduction in time-series analysis.  We will use 1 minute for the analysis below.

###Single-day processing
First, define the standard daily processing steps.

In [ ]:
# define standard segment processing
w = scipy.signal.get_window('hann',sample_rate)
totsec = (v.size)/sample_rate # total number of seconds in vector
spa = 60  # seconds per average
nseg = int(totsec/spa)
print(f'{nseg} segments of length {spa} seconds in {totsec} seconds of audio')

# initialize empty spectrogram
nfreq = int(sample_rate/2+1)
sg = np.empty((nfreq, nseg), float)
print(f':: individual spectrogram shape   =   {sg.shape}')

# process
for x in range(0,nseg):
  cstart = x*spa*sample_rate; cend = (x+1)*spa*sample_rate
  cv = v[cstart:cend]
  f,psd = scipy.signal.welch(cv,fs=sample_rate,window=w,nfft=sample_rate)
  psd = 10*np.log10(psd) - sens
  sg[:,x] = psd

# plot spectrogram for example day
plt.figure(dpi=200, figsize = [9,3])
plt.imshow(sg,aspect='auto',origin='lower',vmin=45,vmax=105)
plt.yscale('log')
plt.ylim(10,1000)
plt.colorbar(label='Spectrum level (dB re 1 $\mu$Pa$^2$/Hz)')
plt.xlabel('Minute of day')
plt.ylabel('Frequency (Hz)')
plt.title('01-November-2017')
plt.annotate("B$_3$",(10,41),color='w')

In the frequency band of the B-call third harmonic, just above 40 Hz, we can see calls present throughout the day, with greater intensity during the first half of the day.  This is more clearly quantified by CI.

In [ ]:
# compute call index for the day at 1-minute resolution
pk = np.squeeze(sg[f==43,:])
bg = np.mean(np.squeeze(np.array(sg[(f==35) | (f==50),:])),axis=0); 
CI = pk/bg; 
# compute hourly means
hr = np.arange(0, 1440, 1)/60    # hour of day reference
bin_means, bin_edges, binnumber = scipy.stats.binned_statistic(hr, CI, statistic='mean', bins=24, range=(0,24))
hourcenters = np.arange(0.5,24.5,1)
# plot
plt.figure(dpi=200, figsize = [9,3])
plt.plot(hr,CI,'.',color='lightgray')
plt.plot(hourcenters,bin_means,'ks')
plt.xlabel('Hour of day')
plt.ylabel('B-call index (CI)')
plt.xlim(0,24)
_ = plt.xticks(np.arange(0,25,1))
plt.title('CI at 1-minute and hourly resolution')

### Batch processing
To consider variability at longer time scales, let's process one week of data, computing CI for each minute of each day.  The week of November 12 - 18, 2017 is a good week, with strong variation in the call index and no recording gaps.

In [ ]:
# Batch process the week of daily files
F = -1   # first row index will be 0 after increment
spa = 60  # seconds per average
nseg = int(86400/spa)   # number of processing segments
# Initialize arrays to hold CI
# dimensions: number of days x number of segments per day
NumberOfDays = 7
CIm = np.zeros((NumberOfDays,nseg))

sample_rate = 2000   # data sample rate
w = scipy.signal.get_window('hann',sample_rate)   # 1-second window for 2 kHz data
nfreq = int(sample_rate/2+1)   # number of output frequencies in spectrogram

D = range(12,19)
for n in D:
  filename = "MARS-201711" + str(n) + "T000000Z-2kHz.wav"
  key = f'{year}/{month}/{filename}'
  url = f'https://{bucket}.s3.amazonaws.com/{key}'
  print(f'Reading {url}') 
  F = F+1
  v, fs = sf.read(io.BytesIO(urlopen(url).read()),dtype='float32')
  v = v*3   # convert scaled voltage to volts

  # initialize empty spectrogram
  sg = np.empty((nfreq, nseg), float)

  # process
  for x in range(0,nseg):
    cstart = x*spa*fs; cend = (x+1)*spa*fs; cv = v[cstart:cend]
    f,psd = scipy.signal.welch(cv,fs,window=w,nfft=fs)
    psd = 10*np.log10(psd) - sens
    sg[:,x] = psd
  # CI
  pk = np.squeeze(sg[f==43,:])
  bg = np.mean(np.squeeze(np.array(sg[(f==35) | (f==50),:])),axis=0); 
  CI = pk/bg;
  CIm[F,:] = CI

View the array containing 1-minute CI for the 7 days.

In [ ]:
plt.figure(dpi=200, figsize = [9,3])
plt.imshow(CIm,aspect='auto',origin='lower')
plt.xlabel('Minute of day')
plt.ylabel('Day of week')
plt.title('Blue whale B call index')
plt.colorbar()

This time series shows strong variations on short time time scales.  Let's represent the time series as a boxplot with daily bins.

In [ ]:
capprops = dict(linewidth=1, color='gray')
whiskerprops = dict(linewidth=1, color='gray')
medianprops = dict(linewidth=1, color='gray')
boxprops = dict(linewidth=1, color='gray')
plt.figure(dpi=200, figsize = [2,4])
plt.boxplot(np.transpose(CIm), showfliers=False, notch=True, patch_artist=True, boxprops = boxprops, capprops=capprops, whiskerprops=whiskerprops, medianprops=medianprops)
plt.ylabel('Call Index')
plt.xlabel('Day of week')

There is also significant day-to-day variation during this week, which shows a rise, plateau, and fall in CI values.  To examine variation in the B-call index relative to the counts of A calls that we will obtain using supervised machine learning (next tutorial), an effective temporal resolution will be hourly.

In [ ]:
# compute hourly mean B-call index
bin_means, bin_edges, binnumber = scipy.stats.binned_statistic(hr, CIm, statistic='mean', bins=24, range=(0,24))
hourlyCI = bin_means.flatten()
tsday = np.arange(0, 7, 1/24)    # fractional day reference
plt.figure(dpi=200, figsize = [9,3])
plt.plot(tsday,hourlyCI,'ks',markersize=3)
plt.ylabel('Hourly CI')
plt.xlabel('Days')

Save the hourly results.

In [ ]:
with open('HourlyCI-12-18Nov2017.txt', 'w') as fileout:
    for line in hourlyCI:
        fileout.write(str(line))
        fileout.write('\n')
fileout.close()

## References
---
Širović, A., J.A. Hildebrand, S.M. Wiggins, D. Thiele (2009) Blue and fin whale acoustic presence around Antarctica during 2003 and 2004
Mar. Mamm. Sci., 25, pp. 125-136

Širović, A., J. A. Hildebrand, S. M. Wiggins, M. A. McDonald, S. E. Moore, D. Thiele (2004) Seasonality of blue and fin whale calls and the influence of sea ice in the Western Antarctic Peninsula. Deep-Sea Research II 51:2327–2344.

Oestreich, W.K., J.A. Fahlbusch, D.E. Cade, J. Calambokidis, T. Margolina, J. Joseph, A.S. Friedlaender, M.F. McKenna, A.K. Stimpert, B.L. Southall, J.A. Goldbogen, J.P. Ryan (2020). Animal-borne metrics enable acoustic detection of blue whale migration. Current Biology 30(23), 4773-4779.